In [18]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.cross_validation import train_test_split
import tflearn
import joblib

In [2]:
X = np.arange(100)
X = X.reshape(-1,1)
std = 25
y = X * 5 + np.random.normal(0, std, X.shape)

In [3]:
X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.3)

In [4]:
l0 = tflearn.input_data(shape=[None,1])
l1 = tflearn.fully_connected(l0, 100, activation='relu')
l2 = tflearn.fully_connected(l1, 1)
net = tflearn.regression(l2, loss="mean_square", restore=False)

In [5]:
model = tflearn.DNN(net)

In [6]:
model.fit(X_tr, y_tr, n_epoch=500)

Training Step: 1000  | total loss: 745.65228
| Adam | epoch: 500 | loss: 745.65228 -- iter: 70/70
Training Step: 1000  | total loss: 745.65228
| Adam | epoch: 500 | loss: 745.65228 -- iter: 70/70
--


In [17]:
# saving model
data = {
    'l1W':model.get_weights(l1.W),
    'l1b':model.get_weights(l1.b),
    'l2W':model.get_weights(l2.W),
    'l2b':model.get_weights(l2.b),
}
joblib.dump(data, 'model.tfl', compress=3)
# loading model
data_loaded = joblib.load('model.tfl')
model_loaded = tflearn.DNN(net)
model_loaded.set_weights(l1.W, data_loaded['l1W'])
model_loaded.set_weights(l1.b, data_loaded['l1b'])
model_loaded.set_weights(l2.W, data_loaded['l2W'])
model_loaded.set_weights(l2.b, data_loaded['l2b'])
y_pred = model.predict(X_t)
mae = mean_absolute_error(y_pred, y_t)
r2 = r2_score(y_pred, y_t)
print mae, r2

16.4513112952 0.973639519993


In [13]:
m2 = tflearn.DNN(net)

In [16]:
y_pred2 = m2.predict(X_t)
mae = mean_absolute_error(y_pred2, y_t)
r2 = r2_score(y_pred2, y_t)
print mae, r2

16.4513112952 0.973639519993
